In [1]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [2]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv', 
                'nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 'nb_conv_layers', 'nb_bn_layers',
                'nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()
train_dataset.tail()

,Model_name,input_shape,power,input_width,input_size,nb_params_conv,nb_params_bn,nb_params_fc,flops,nb_layers,nb_conv_layers,nb_bn_layers,nb_fc_layers,sum_activations,weighted_sum_neurons
1521,basic_model_320_23,"(1, 320, 320, 3)",15471.0,320,307200,2499431,7992,5701,168678951282,44,23,15,2,298643104,84243523392
1522,basic_model_600_42,"(1, 600, 600, 3)",13356.0,600,1080000,12220342,14132,25701,159725668606,71,42,20,2,140162512,79782663040
1523,basic_model_320_20,"(1, 320, 320, 3)",25720.0,320,307200,8674305,9112,1211,1776131430777,37,20,14,2,629248140,887853056000
1524,basic_model_600_31,"(1, 600, 600, 3)",23914.0,600,1080000,12481366,9088,33201,853243940125,52,31,13,2,337941414,426431945716
1525,basic_model_75_12,"(1, 75, 75, 3)",2132.0,75,16875,1179844,2588,28501,3330331266,24,12,6,2,5411083,1656063392


In [32]:
# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'power', 'input_width', 'input_size', 'nb_params_conv', 
                'nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 'nb_conv_layers', 'nb_bn_layers',
                'nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset = raw_dataset.copy()
test_dataset.isna().sum()
test_dataset = test_dataset.dropna()
test_dataset.tail()

,Model_name,input_shape,power,input_width,input_size,nb_params_conv,nb_params_bn,nb_params_fc,flops,nb_layers,nb_conv_layers,nb_bn_layers,nb_fc_layers,sum_activations,weighted_sum_neurons
102,EfficientNet-B0,"(512, 512, 3)",4041.0,512,786432,3965532,84033,1281001,4085726980,233,81,50,2,117635172,80094501376
103,EfficientNet-B0,"(720, 720, 3)",5866.0,720,1555200,3965532,84033,1281001,8183072596,233,81,50,2,233556068,161236938752
104,EfficientNet-B4,"(90, 90, 3)",2136.0,90,24300,17423416,250401,1793001,595640786,470,160,97,2,10284484,12812090224
105,EfficientNet-B4,"(512, 512, 3)",6704.0,512,786432,17423416,250401,1793001,15877844034,470,160,97,2,305708296,366198974816
106,EfficientNet-B4,"(720, 720, 3)",10044.0,720,1555200,17423416,250401,1793001,31863479154,470,160,97,2,607426440,737567732576


In [4]:
train_dataset.pop('input_width')
train_dataset.pop('nb_params_bn')
train_dataset.pop('nb_params_fc')
train_dataset.pop('nb_conv_layers')
train_dataset.pop('nb_bn_layers')
train_dataset.pop('nb_fc_layers')

0       1
1       1
2       1
3       1
4       1
       ..
1521    2
1522    2
1523    2
1524    2
1525    2
Name: nb_fc_layers, Length: 1525, dtype: int64

In [33]:
test_dataset.pop('input_width')
test_dataset.pop('nb_params_bn')
test_dataset.pop('nb_params_fc')
test_dataset.pop('nb_conv_layers')
test_dataset.pop('nb_bn_layers')
test_dataset.pop('nb_fc_layers')

0      2
1      2
2      2
3      2
4      2
      ..
102    2
103    2
104    2
105    2
106    2
Name: nb_fc_layers, Length: 107, dtype: int64

In [6]:
Y = train_dataset.pop('power')
X = train_dataset
Y_f = Y
X_f = X

In [7]:
X.tail()

,Model_name,input_shape,input_size,nb_params_conv,flops,nb_layers,sum_activations,weighted_sum_neurons
1521,basic_model_320_23,"(1, 320, 320, 3)",307200,2499431,168678951282,44,298643104,84243523392
1522,basic_model_600_42,"(1, 600, 600, 3)",1080000,12220342,159725668606,71,140162512,79782663040
1523,basic_model_320_20,"(1, 320, 320, 3)",307200,8674305,1776131430777,37,629248140,887853056000
1524,basic_model_600_31,"(1, 600, 600, 3)",1080000,12481366,853243940125,52,337941414,426431945716
1525,basic_model_75_12,"(1, 75, 75, 3)",16875,1179844,3330331266,24,5411083,1656063392


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.001, random_state=42, shuffle=True)

In [9]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

In [10]:
X_test_model_names = X_test.pop('Model_name')
X_test_input_shapes = X_test.pop('input_shape')

In [11]:
X_f_model_names = X_f.pop('Model_name')
X_f_input_shapes = X_f.pop('input_shape')
dtrain_f = xgb.DMatrix(X_f, label=Y_f)

In [12]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [34]:
Y_e = test_dataset.pop('power')
X_e = test_dataset
X_e.tail()
X_test_e_model_names = X_e.pop('Model_name')
X_test_e_input_shapes = X_e.pop('input_shape')
dtest_e = xgb.DMatrix(X_e, label=Y_e)

In [14]:
params = {
    # Parameters that we are going to tune.
    'max_depth':17,
    'min_child_weight': 14,
    'eta':.08,
    'subsample': 1,
    'colsample_bytree': .9,
    'gamma' : 0.7,
    # Other parameters
    'objective':'reg:squarederror',
}

In [15]:
params['eval_metric'] = "rmse"
num_boost_round = 3000

In [16]:
print(params)

{'max_depth': 17, 'min_child_weight': 14, 'eta': 0.08, 'subsample': 1, 'colsample_bytree': 0.9, 'gamma': 0.7, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}


In [17]:
num_boost_round = 98
best_model = xgb.train( 
    params,
    dtrain_f,
    num_boost_round=num_boost_round
)

In [18]:
#best_model.save_model("model_16-04.model")
loaded_model = best_model

In [35]:
y_pred = loaded_model.predict(dtest_e)
np_test = Y_e.values
y_test = Y_e.values

In [36]:
# Model evaluation on test dataset
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error :', median_absolute_error(y_test, y_pred))

Explained variance :  0.9915254766007869
R2_value :  0.9914098180950458
Mean absolute error : 286.2516930125584
Root Mean squared error : 439.131836379402
Median absolute error : 180.623046875


In [37]:
# Calculate mean absolute percentage error (MAPE)
np_test = y_test
errors = abs(y_pred-np_test)
mape = 100 * (errors / np_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE : ',max(mape))
print('Mape:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE :  34.138259566862914
Mape: 4.51 %.
Accuracy: 95.49 %.


In [38]:
test_models = X_test_e_model_names.values
test_input_shapes = X_test_e_input_shapes.values

In [39]:
import csv
with open('./measured_vs_predicted_XGBoost_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, np_test[i], y_pred[i],abs(y_pred[i]-np_test[i]), ((abs(y_pred[i]-np_test[i]))/np_test[i])*100])